In [ ]:
import pandas as pd
import pymc_marketing as pmm
import matplotlib as mpl
import matplotlib.pyplot as plt
from cabm import post_processing as ppr

In [ ]:
# Figure setup
# Set the font to Courier
mpl.rcParams['font.family'] = 'Courier'

# Set the line weight to thin
mpl.rcParams['lines.linewidth'] = 0.75

# Set the color scheme to black and white
mpl.rcParams['text.color'] = 'black'
mpl.rcParams['axes.labelcolor'] = 'black'
mpl.rcParams['xtick.color'] = 'black'
mpl.rcParams['ytick.color'] = 'black'
mpl.rcParams['axes.edgecolor'] = 'black'

In [ ]:
agent_df = pd.read_pickle('agent_output_A_ADS_INCR.pkl')

In [ ]:
agent_df = ppr.add_date_column(agent_df)

In [ ]:
data_A = ppr.calculate_total_purchases(agent_df, ['price_A','tv_A','web_A', 'price_B', 'tv_B', 'web_B'])


In [ ]:
data_A = ppr.add_total_sales_columns(data_A)

In [ ]:
data_A = ppr.ensure_float_columns(data_A, exclude_columns=['Date','Step'])

In [ ]:
data_a_mod = data_A[52:].reset_index(drop=True)

In [ ]:
data_a_mod['Total_Sales_A'] = data_a_mod['Total_Sales_A'] * 1000
data_a_mod['Total_Sales_B'] = data_a_mod['Total_Sales_B'] * 1000

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(data_a_mod['Date'], data_a_mod['Total_Sales_A'],color='black')
plt.xlabel('Date')
plt.ylabel('Total Sales A')
plt.title('Total Sales A vs Date')
plt.show()


In [ ]:
data_a_mod.to_csv("sim_data_trunc.csv", index=False)

In [ ]:
# Define the columns
date_column = 'Date'
channel_columns = ['tv_A', 'web_A']
control_columns = ['Total_Sales_B']

my_sampler_config = {
    'tune': 1000,
    'draws': 1000,
    'chains': 4,
    #'target_accept': 0.9  # Increase this value to increase target_accept
}

# Instantiate the model
mmm = pmm.mmm.delayed_saturated_mmm.DelayedSaturatedMMM(
    sampler_config=my_sampler_config,
    date_column=date_column,
    channel_columns=channel_columns,
    control_columns=control_columns,
    adstock_max_lag=8,
)

In [ ]:
# Split data into predictors (X) and target (y)
X = data_a_mod.drop('Total_Sales_A', axis=1)
y = data_a_mod['Total_Sales_A']

# Fit the model
mmm.fit(X, y)

In [ ]:
import arviz as az
import pymc as pm
import seaborn as sns

In [ ]:
mmm.fit_result

In [ ]:
az.summary(
    data=mmm.fit_result,
    var_names=[
        "intercept",
        "likelihood_sigma",
        "beta_channel",
        "alpha",
        "lam",
    ],
)

In [ ]:
_ = az.plot_trace(
    data=mmm.fit_result,
    var_names=[
        "intercept",
        "likelihood_sigma",
        "beta_channel",
        "alpha",
        "lam",
    ],
    compact=True,
    backend_kwargs={"figsize": (12, 10), "layout": "constrained"},
)

In [ ]:
mmm.sample_posterior_predictive(X, extend_idata=True, combined=True)

In [ ]:
mmm.plot_posterior_predictive(original_scale=True);

In [ ]:
mmm.plot_components_contributions()

In [ ]:
mmm.plot_waterfall_components_decomposition()

In [ ]:
import pprint

In [ ]:
dummy_model = DelayedSaturatedMMM(date_column="", channel_columns="", adstock_max_lag=4)
pprint.pprint(dummy_model.default_model_config)